In [15]:
import os
import pandas as pd

In [16]:
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
cln_pkl_loc = os.path.join(ROOT_DIR, 'data_cleaning','cleanweathersmall.pkl')

In [17]:
df = pd.read_pickle(cln_pkl_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3765246 entries, 0 to 3765245
Data columns (total 9 columns):
 #   Column   Dtype         
---  ------   -----         
 0   station  object        
 1   time     datetime64[ns]
 2   temp     float64       
 3   dwpt     float64       
 4   rhum     float64       
 5   prcp     float64       
 6   wdir     float64       
 7   wspd     float64       
 8   pres     float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 258.5+ MB


In [18]:
pivoted_df = df.pivot(index='time', columns='station', values=['temp', 'dwpt','rhum','prcp','wdir','wspd','pres'])
pivoted_df.columns = ['_'.join(col) for col in pivoted_df.columns.values]
pivoted_df

,temp_0CNUO,temp_0CO7B,temp_0FV1F,temp_1J1PJ,temp_1JWST,temp_20QWH,temp_21O3U,temp_2W8UZ,temp_3S56J,temp_4DUJO,...,pres_W5F5F,pres_X9FED,pres_XM44W,pres_Y59TE,pres_Z7ZOG,pres_ZFZUV,pres_ZNWZW,pres_ZUQJS,pres_ZWC6W,pres_ZYITU
time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,-25.10,5.00,1.20,-0.90,-6.50,-28.70,-29.00,4.60,-26.50,1.80,...,NaN,NaN,1013.50,NaN,NaN,1023.80,NaN,NaN,1010.80,NaN
2022-01-01 01:00:00,-25.70,5.10,0.90,-0.60,-7.50,-29.50,-28.90,4.60,-27.30,1.60,...,NaN,NaN,1014.60,NaN,NaN,1024.20,NaN,NaN,1011.80,NaN
2022-01-01 02:00:00,-26.20,4.20,1.50,-0.80,-8.60,-30.20,-30.10,4.70,-27.90,0.90,...,NaN,NaN,1015.10,NaN,NaN,1024.90,NaN,NaN,1012.70,NaN
2022-01-01 03:00:00,-26.10,5.50,4.10,-0.90,-9.40,-29.90,-30.10,4.60,-28.10,1.00,...,NaN,NaN,1016.00,NaN,NaN,1025.40,NaN,NaN,1013.60,NaN
2022-01-01 04:00:00,-26.50,5.90,3.10,-1.20,-10.10,-30.40,-31.40,4.90,-28.60,0.50,...,NaN,NaN,1017.30,NaN,NaN,1025.40,NaN,NaN,1013.90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-22 19:00:00,12.40,17.80,13.70,11.50,10.20,13.50,12.50,15.20,10.80,12.40,...,1022.10,1025.20,1022.60,1025.00,1025.30,1024.50,1025.40,1022.60,1017.80,1016.70
2022-09-22 20:00:00,13.00,17.00,14.30,12.00,10.40,14.00,12.50,14.80,11.40,11.60,...,1020.30,1024.40,1022.40,1024.00,1024.00,1023.10,1025.00,1022.20,1017.90,1016.00
2022-09-22 21:00:00,13.20,16.60,13.90,12.40,10.50,14.30,12.40,15.10,11.70,10.80,...,1019.60,1024.20,1022.20,1023.00,1023.70,1022.00,1024.00,1022.40,1018.10,1017.40


### Our target is Ann Arbor which is station KARB0. This simple model will just try to predict the temperature for KARB0. So will build some features specifically around it.

In [19]:
ann_arbor_cols = [col for col in pivoted_df.columns if "KARB0" in col]
ann_arbor_df = pivoted_df[ann_arbor_cols].copy()

#### This adds a columnn for the weather at 24 hrs prior to our target

In [20]:
for col in ann_arbor_df.columns:
    ann_arbor_df[f'24 hr~{col}'] = ann_arbor_df[col].shift(-24)

In [21]:
ann_arbor_df = ann_arbor_df.rename_axis(None, axis = 0)
ann_arbor_df = ann_arbor_df[ann_arbor_df['24 hr~temp_KARB0'].notna()]
ann_arbor_df

,temp_KARB0,dwpt_KARB0,rhum_KARB0,prcp_KARB0,wdir_KARB0,wspd_KARB0,pres_KARB0,24 hr~temp_KARB0,24 hr~dwpt_KARB0,24 hr~rhum_KARB0,24 hr~prcp_KARB0,24 hr~wdir_KARB0,24 hr~wspd_KARB0,24 hr~pres_KARB0
2022-01-01 00:00:00,5.00,2.90,86.00,0.00,120.00,11.00,1005.00,-2.20,-3.90,88.00,0.60,50.00,16.60,1010.00
2022-01-01 01:00:00,5.00,2.90,86.00,0.00,120.00,11.20,1005.40,-2.80,-5.60,81.00,0.70,30.00,16.60,1010.20
2022-01-01 02:00:00,5.00,2.90,86.00,0.00,120.00,5.40,1005.30,-3.30,-5.60,84.00,0.40,30.00,24.10,1009.90
2022-01-01 03:00:00,5.00,2.90,86.00,0.00,0.00,0.00,1004.90,-3.90,-6.10,85.00,0.30,20.00,18.40,1010.30
2022-01-01 04:00:00,5.60,3.30,85.00,0.00,0.00,0.00,1004.90,-4.40,-6.10,88.00,0.30,30.00,16.60,1010.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-21 19:00:00,29.40,20.60,59.00,0.00,290.00,22.30,1007.30,15.00,3.50,46.00,0.00,320.00,25.90,1017.60
2022-09-21 20:00:00,28.30,20.10,61.00,0.00,260.00,18.40,1007.00,16.10,4.50,46.00,0.00,320.00,18.40,1017.90
2022-09-21 21:00:00,27.80,19.30,60.00,0.00,280.00,18.40,1007.40,16.00,4.40,46.00,0.00,330.00,22.00,1019.00
2022-09-21 22:00:00,26.70,18.80,62.00,0.00,250.00,16.60,1007.40,14.00,2.90,47.00,0.00,330.00,22.00,1019.00


### Now our target will be the '-24hr~temp_KARB0' column, which is the temperature 24 hrs in the future from all of the original features.

In [14]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state =696)
reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None)
models,predictions = reg.fit(X_train, X_test, y_train, y_test)
print(models)

 24%|██▍       | 10/42 [00:01<00:04,  7.10it/s]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfGammaLoss'.


 67%|██████▋   | 28/42 [00:09<00:06,  2.30it/s]

PoissonRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfPoissonLoss'.


100%|██████████| 42/42 [20:18<00:00, 29.02s/it] 

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
ExtraTreesRegressor                          0.93       0.93   3.30   
RandomForestRegressor                        0.92       0.92   3.34   
XGBRegressor                                 0.91       0.91   3.56   
BaggingRegressor                             0.91       0.91   3.57   
HistGradientBoostingRegressor                0.91       0.91   3.59   
LGBMRegressor                                0.91       0.91   3.59   
KNeighborsRegressor                          0.90       0.90   3.87   
GradientBoostingRegressor                    0.89       0.89   4.01   
MLPRegressor                                 0.89       0.89   4.07   
SVR                                          0.88       0.88   4.29   
NuSVR                                        0.87       0.87   4.33   
Lars                                         0.87       0.87   4.45   
LarsCV

In [27]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.dummy import DummyRegressor
X_cols = [col for col in ann_arbor_df.columns if "~" not in col]
X = ann_arbor_df[X_cols]
y = ann_arbor_df['24 hr~temp_KARB0']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
xt_reg = ExtraTreesRegressor(n_estimators=100, random_state=696,n_jobs=-1).fit(X_train, y_train)
lasso_reg = linear_model.Lasso(alpha=0.1).fit(X_train, y_train)
tw_reg = linear_model.TweedieRegressor().fit(X_train, y_train)
dummy_reg = DummyRegressor(strategy="mean").fit(X_train, y_train)

In [28]:
print(f"Extra Trees Regressor acc score: {xt_reg.score(X_test,y_test)}")
print(f"      Lasso Regressor acc score: {lasso_reg.score(X_test,y_test)}")
print(f"    Tweedie Regressor acc score: {tw_reg.score(X_test,y_test)}")
print(f"      Dummy Regressor acc score: {dummy_reg.score(X_test,y_test)}")

Extra Trees Regressor acc score: 0.9328335846236752
      Lasso Regressor acc score: 0.85775179782658
    Tweedie Regressor acc score: 0.8461027179788403
      Dummy Regressor acc score: -0.0010437921892019197


In [30]:
feature_importance_df = pd.DataFrame([X.columns, xt_reg.feature_importances_]).transpose()
feature_importance_df.columns = ['Feature', 'Importance']
feature_importance_df

,Feature,Importance
0,temp_KARB0,0.47
1,dwpt_KARB0,0.38
2,rhum_KARB0,0.04
3,prcp_KARB0,0.01
4,wdir_KARB0,0.03
5,wspd_KARB0,0.02
6,pres_KARB0,0.04


In [33]:
import altair as alt
alt.Chart(feature_importance_df).mark_bar().encode(
    x=alt.X('Importance:Q', axis=alt.Axis(format="%", tickSize=0, labelFontSize=12)),
    y=alt.Y(
        'Feature:N', sort=list(feature_importance_df.Feature), title="",
        axis=alt.Axis(tickSize=0, labelFontSize=12, labelPadding=10)),
).properties(
    height=600
)



alt.Chart(...)